In [1]:
!pip install -q ultralytics thop pycocotools pandas matplotlib seaborn faster_coco_eval
!pip install -q "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 579.2/579.2 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
%cd /kaggle/working
!rm -rf /kaggle/working/RT-CO-DETR 
!git clone https://github.com/nam-htran/RT-CO-DETR

/kaggle/working
Cloning into 'RT-CO-DETR'...
remote: Enumerating objects: 330, done.
remote: Counting objects: 100% (330/330), done.
remote: Compressing objects: 100% (235/235), done.
remote: Total 330 (delta 119), reused 301 (delta 90), pack-reused 0 (from 0)
Receiving objects: 100% (330/330), 229.57 KiB | 6.56 MiB/s, done.
Resolving deltas: 100% (119/119), done.


In [3]:
%%writefile /kaggle/working/RT-CO-DETR/final_benchmark.py
import os
import sys
import subprocess
import torch
import warnings
import yaml
import re
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from pathlib import Path
from tqdm import tqdm
import json
import shutil

# --- Cấu hình các đường dẫn CỐ ĐỊNH ---
RTDETR_REPO_PATH = Path("/kaggle/working/RT-CO-DETR")
sys.path.insert(0, str(RTDETR_REPO_PATH / "rtdetr"))
sys.path.insert(0, str(RTDETR_REPO_PATH))

BASE_INPUT_PATH = Path("/kaggle/input/rt-co-detr-trained/RT-CO-DETR")
DATA_PATH = Path("/kaggle/input/dsp-pre-final/processed_taco_coco")
OUTPUT_PATH = Path("/kaggle/working/benchmark_output")
YOLO_DATA_PATH = OUTPUT_PATH / "taco_yolo_for_eval"
OUTPUT_PATH.mkdir(exist_ok=True)
YOLO_DATA_PATH.mkdir(exist_ok=True)

try:
    from ultralytics import YOLO
    from thop import profile
    from src.core import YAMLConfig
except ImportError as e:
    print(f"Lỗi import: {e}.")
    sys.exit(1)

# --- Định nghĩa các mô hình ---
MODELS_TO_BENCHMARK = OrderedDict({
    "RT-DETR (Distilled)": { "type": "rtdetr", "weights": BASE_INPUT_PATH / "output/FINETUNE_DISTILLED/best.pth", "config_template": BASE_INPUT_PATH / "templates/rtdetrv2_taco_finetune_distilled.yml.template", "generated_config": OUTPUT_PATH / "flat_distilled.yml" },
    "RT-DETR (Baseline)": { "type": "rtdetr", "weights": BASE_INPUT_PATH / "output/FINETUNE_BASELINE/best.pth", "config_template": BASE_INPUT_PATH / "templates/rtdetrv2_taco_finetune_baseline.yml.template", "generated_config": OUTPUT_PATH / "flat_baseline.yml" },
    "YOLOv11l (Baseline)": { "type": "yolo", "weights": BASE_INPUT_PATH / "output/YOLO/yolo_checkpoints/yolo11l_finetune_baseline/weights/best.pt", "config": YOLO_DATA_PATH / "taco_yolo.yaml" }
})

# --- Các hàm tiện ích ---
def print_header(title):
    print("\n" + "="*85 + f"\n| {title:^81} |\n" + "="*85)

def run_command(command, working_dir=None):
    print(f"\n---> Đang thực thi: {' '.join(command)}")
    full_output = []
    try:
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, encoding='utf-8', cwd=working_dir)
        for line in iter(process.stdout.readline, ''): print(line.strip()); full_output.append(line.strip())
        process.wait()
        print(f"--- Lệnh {'thành công' if process.returncode == 0 else f'thất bại với mã lỗi {process.returncode}'} ---")
        return process.returncode, "\n".join(full_output)
    except Exception as e:
        print(f"Lỗi khi thực thi lệnh: {e}"); return -1, str(e)

def parse_rtdetr_output(output_text):
    metrics = {}
    try:
        ap_50_95 = re.search(r"Average Precision\s+\(AP\) @\[ IoU=0.50:0.95 .* = ([\d\.]+)", output_text)
        ap_50 = re.search(r"Average Precision\s+\(AP\) @\[ IoU=0.50\s+.* = ([\d\.]+)", output_text)
        if ap_50_95: metrics['mAP50-95'] = float(ap_50_95.group(1))
        if ap_50: metrics['mAP50'] = float(ap_50.group(1))
    except Exception as e: print(f"Không thể trích xuất chỉ số từ output RT-DETR: {e}")
    return metrics

def merge_dicts(base, new):
    for key, value in new.items():
        if key in base and isinstance(base.get(key), dict) and isinstance(value, dict):
            merge_dicts(base[key], value)
        else:
            base[key] = value
    return base

def find_include_file(filename, search_dir):
    for root, _, files in os.walk(search_dir):
        if filename in files:
            return Path(root) / filename
    return None

def flatten_yaml_config_recursive(config_path, repo_config_root):
    if not Path(config_path).exists():
        print(f"Cảnh báo: Không tìm thấy file config '{config_path}'")
        return {}
    
    with open(config_path, 'r') as f: config = yaml.safe_load(f) or {}
    if '__include__' in config:
        includes = config.pop('__include__')
        base_config = {}
        if isinstance(includes, str): includes = [includes]
        for include_relative in includes:
            include_filename = Path(include_relative).name
            found_path = find_include_file(include_filename, repo_config_root)
            if found_path:
                included_data = flatten_yaml_config_recursive(found_path, repo_config_root)
                base_config = merge_dicts(base_config, included_data)
            else:
                 print(f"Cảnh báo: Không tìm thấy file include '{include_relative}' trong '{repo_config_root}'")
        return merge_dicts(base_config, config)
    return config

def generate_rtdetr_configs():
    print("--- Tạo file config RT-DETR từ template ---")
    repo_config_root = RTDETR_REPO_PATH / "rtdetr/configs"
    for name, model_info in MODELS_TO_BENCHMARK.items():
        if model_info['type'] == 'rtdetr':
            template_path, output_path = model_info['config_template'], model_info['generated_config']
            flat_config = flatten_yaml_config_recursive(template_path, repo_config_root)
            data_updates = {
                'val_dataloader': {'dataset': {'img_folder': str(DATA_PATH / 'val2017'), 'ann_file': str(DATA_PATH / 'annotations/instances_val2017.json')}},
                'train_dataloader': {'dataset': {'img_folder': str(DATA_PATH / 'train2017'), 'ann_file': str(DATA_PATH / 'annotations/instances_train2017.json')}},
            }
            flat_config = merge_dicts(flat_config, data_updates)
            
            # <<< SỬA LỖI: Đặt output_dir thành một chuỗi đơn giản >>>
            flat_config['output_dir'] = str(OUTPUT_PATH / f"temp_run_{name.replace(' ', '_')}")
            
            flat_config.pop('tuning', None)

            with open(output_path, 'w') as f: yaml.dump(flat_config, f, sort_keys=False, default_flow_style=False)
            print(f"Đã tạo file config (flattened) cho '{name}' tại: {output_path}")

def prepare_yolo_dataset_for_eval():
    print_header("Chuẩn bị dữ liệu cho YOLO Evaluation")
    val_images_dir, val_labels_dir = YOLO_DATA_PATH / "images/val", YOLO_DATA_PATH / "labels/val"
    if val_labels_dir.exists() and any(val_labels_dir.iterdir()):
        print("...Dữ liệu YOLO đã tồn tại, bỏ qua bước chuẩn bị.")
        return
    val_images_dir.mkdir(parents=True, exist_ok=True); val_labels_dir.mkdir(parents=True, exist_ok=True)
    print("... Sao chép ảnh validation"); [shutil.copy(f, val_images_dir) for f in tqdm((DATA_PATH / "val2017").glob("*.jpg"))]
    print("... Chuyển đổi annotations sang định dạng YOLO")
    with open(DATA_PATH / "annotations/instances_val2017.json") as f: data = json.load(f)
    images_info = {img['id']: img for img in data['images']}
    for ann in tqdm(data['annotations']):
        if (image_id := ann['image_id']) not in images_info: continue
        img_info = images_info[image_id]; img_w, img_h = img_info['width'], img_info['height']
        x, y, w, h = ann['bbox']; x_c, y_c, nw, nh = (x+w/2)/img_w, (y+h/2)/img_h, w/img_w, h/img_h
        with open(val_labels_dir / (Path(img_info['file_name']).stem + ".txt"), 'a') as f:
            f.write(f"{ann['category_id']} {x_c:.6f} {y_c:.6f} {nw:.6f} {nh:.6f}\n")
    with open(DATA_PATH / "annotations/instances_train2017.json") as f: coco_data = json.load(f)
    class_names = [cat['name'] for cat in sorted(coco_data['categories'], key=lambda x: x['id'])]
    yolo_yaml_path = MODELS_TO_BENCHMARK["YOLOv11l (Baseline)"]["config"]
    with open(yolo_yaml_path, 'w') as f: yaml.dump({'path': str(YOLO_DATA_PATH.absolute()), 'train': 'images/val', 'val': 'images/val', 'nc': len(class_names), 'names': class_names}, f, sort_keys=False)
    print(f"Đã tạo file YOLO YAML tại: {yolo_yaml_path}")
    
def setup_and_verify_paths():
    print_header("Bước 1: Chuẩn bị và Kiểm tra Files")
    generate_rtdetr_configs()
    prepare_yolo_dataset_for_eval()
    found_paths = OrderedDict()
    for name, path_dict in MODELS_TO_BENCHMARK.items():
        config_path = path_dict.get('generated_config', path_dict.get('config'))
        if path_dict['weights'].exists() and config_path.exists():
            print(f"[SẴN SÀNG] {name}")
            found_paths[name] = path_dict
        else:
            print(f"[BỎ QUA] {name} - Thiếu file.")
            if not path_dict['weights'].exists(): print(f"  -> Thiếu weights: {path_dict['weights']}")
            if not config_path.exists(): print(f"  -> Thiếu config: {config_path}")
    if not found_paths: print("\nKhông có mô hình nào sẵn sàng. Dừng lại."); sys.exit(1)
    print("\n---> Tất cả các file cần thiết đã sẵn sàng!")
    return found_paths

def evaluate_model_accuracy(paths, benchmark_results):
    print_header("Bước 2: Đánh giá Độ chính xác (mAP trên tập Val)")
    for name, path_dict in paths.items():
        print(f"\n--- Đang đánh giá: {name} ---")
        if name not in benchmark_results: benchmark_results[name] = {}
        if path_dict["type"] == "rtdetr":
            command = ["python", str(RTDETR_REPO_PATH / "rtdetr/tools/train.py"), "-c", str(path_dict["generated_config"]), "-r", str(path_dict["weights"]), "--test-only"]
            return_code, output = run_command(command, working_dir=RTDETR_REPO_PATH)
            if return_code == 0: benchmark_results[name].update(parse_rtdetr_output(output))
        elif path_dict["type"] == "yolo":
            model = YOLO(path_dict["weights"])
            results = model.val(data=str(path_dict["config"]), imgsz=640, batch=16, split='val', verbose=False)
            if results and hasattr(results, 'box'):
                benchmark_results[name]['mAP50-95'] = results.box.map
                benchmark_results[name]['mAP50'] = results.box.map50

def analyze_model_complexity(paths, benchmark_results):
    print_header("Bước 3: Phân tích Độ phức tạp (Params & FLOPs)")
    dummy_input = torch.randn(1, 3, 640, 640)
    for name, path_dict in paths.items():
        print(f"\n--- Đang phân tích: {name} ---")
        try:
            model_cpu = None
            config_file = path_dict.get('generated_config', path_dict.get('config'))
            if path_dict["type"] == "rtdetr":
                cfg = YAMLConfig(config_file); model_cpu = cfg.model.cpu().eval()
            elif path_dict["type"] == "yolo":
                model_cpu = YOLO(path_dict["weights"]).model.cpu().eval()
            if model_cpu:
                macs, params = profile(model_cpu, inputs=(dummy_input.cpu(),), verbose=False)
                benchmark_results[name]['Params (M)'] = params / 1e6; benchmark_results[name]['FLOPs (G)'] = macs * 2 / 1e9
                print(f"Params: {params / 1e6:.2f} M, FLOPs: {macs * 2 / 1e9:.2f} G")
        except Exception as e: print(f"Lỗi khi phân tích {name}: {e}")

def measure_inference_speed(paths, benchmark_results):
    print_header("Bước 4: Đo lường Tốc độ Suy luận")
    if not torch.cuda.is_available(): print("!!! CẢNH BÁO: Không tìm thấy GPU. Bỏ qua. !!!"); return
    device = torch.device("cuda"); dummy_input = torch.randn(1, 3, 640, 640, device=device); warmup_runs, timed_runs = 20, 50
    for name, path_dict in paths.items():
        print(f"\n--- Đang đo: {name} ---")
        try:
            model = None
            config_file = path_dict.get('generated_config', path_dict.get('config'))
            if path_dict["type"] == "rtdetr":
                cfg = YAMLConfig(config_file); model = cfg.model
                state_dict = torch.load(path_dict["weights"], map_location=device)
                key = 'ema' if 'ema' in state_dict else 'model'
                state = state_dict.get(key, {}).get('module', state_dict.get(key, state_dict))
                model.load_state_dict(state); model = model.to(device).eval()
            elif path_dict["type"] == "yolo":
                model = YOLO(path_dict["weights"]).model.to(device).eval()
            if model:
                with torch.no_grad():
                    for _ in tqdm(range(warmup_runs), desc="Warm-up", leave=False): _ = model(dummy_input)
                    torch.cuda.synchronize(); start_time = time.time()
                    for _ in tqdm(range(timed_runs), desc="Timing", leave=False): _ = model(dummy_input)
                    torch.cuda.synchronize(); end_time = time.time()
                avg_time_ms = (end_time - start_time) / timed_runs * 1000
                benchmark_results[name]['Speed (ms)'] = avg_time_ms
                print(f"Tốc độ trung bình: {avg_time_ms:.2f} ms/ảnh")
        except Exception as e: print(f"Lỗi khi đo tốc độ {name}: {e}")

def generate_final_summary(benchmark_results):
    print_header("Bảng tổng kết Benchmark")
    df = pd.DataFrame.from_dict(benchmark_results, orient='index')
    column_order = ['mAP50-95', 'mAP50', 'Speed (ms)', 'Params (M)', 'FLOPs (G)']
    df = df.reindex(columns=column_order).dropna(axis=1, how='all')
    for col in ['mAP50-95', 'mAP50']:
        if col in df.columns: df[col] = df[col].map('{:.4f}'.format)
    for col in ['Params (M)', 'FLOPs (G)', 'Speed (ms)']:
        if col in df.columns: df[col] = df[col].map('{:.2f}'.format)
    device_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'
    print(f"Benchmark được thực hiện trên: {device_name}\n")
    print(df.to_string())
    summary_csv_path = OUTPUT_PATH / "benchmark_summary.csv"
    df.to_csv(summary_csv_path); print(f"\nBảng tổng kết đã lưu tại: {summary_csv_path}")
    try:
        df_plot = df.apply(pd.to_numeric, errors='coerce').dropna(subset=['Speed (ms)', 'mAP50-95'])
        if df_plot.empty: print("Không có đủ dữ liệu để vẽ biểu đồ."); return
        plt.style.use('seaborn-v0_8-whitegrid')
        fig, ax = plt.subplots(figsize=(12, 8))
        sizes = (df_plot['Params (M)'] / df_plot['Params (M)'].max() * 600) + 150 if 'Params (M)' in df_plot else 200
        scatter = ax.scatter(df_plot['Speed (ms)'], df_plot['mAP50-95'], s=sizes, c=df_plot.get('FLOPs (G)'), cmap='viridis_r', alpha=0.7, edgecolors="w", linewidth=1.5)
        for i, txt in enumerate(df_plot.index):
            ax.annotate(txt, (df_plot['Speed (ms)'].iloc[i], df_plot['mAP50-95'].iloc[i]), xytext=(10, -10), textcoords='offset points', ha='left', arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=-0.2", color='gray'))
        ax.set_title('So sánh hiệu năng: Tốc độ vs. Độ chính xác', fontsize=16, weight='bold')
        ax.set_xlabel('Thời gian suy luận (ms) - Càng thấp càng tốt', fontsize=12)
        ax.set_ylabel('mAP @ .50-.95 - Càng cao càng tốt', fontsize=12)
        ax.invert_xaxis()
        if 'FLOPs (G)' in df_plot:
            cbar = fig.colorbar(scatter, ax=ax, pad=0.01); cbar.set_label('FLOPs (G)', rotation=270, labelpad=20)
        plot_path = OUTPUT_PATH / "benchmark_plot.png"
        plt.savefig(plot_path, bbox_inches='tight'); print(f"Biểu đồ so sánh đã lưu tại: {plot_path}")
    except Exception as e: print(f"Không thể tạo biểu đồ: {e}")

def main():
    warnings.filterwarnings("ignore", category=UserWarning)
    benchmark_results = OrderedDict()
    paths_to_benchmark = setup_and_verify_paths()
    if paths_to_benchmark:
        evaluate_model_accuracy(paths_to_benchmark, benchmark_results)
        analyze_model_complexity(paths_to_benchmark, benchmark_results)
        measure_inference_speed(paths_to_benchmark, benchmark_results)
    generate_final_summary(benchmark_results)
    print_header("QUY TRÌNH BENCHMARK HOÀN TẤT")

if __name__ == "__main__":
    main()

Writing /kaggle/working/RT-CO-DETR/final_benchmark.py


In [4]:
%cd /kaggle/working/RT-CO-DETR
!python final_benchmark.py

/kaggle/working/RT-CO-DETR
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
E0000 00:00:1761982017.157355      76 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761982017.211110      76 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

|                        Bước 1: Chuẩn bị và Kiểm tra Files                         |
--- Tạo file config RT-DETR từ template ---
Đã tạo file config (flattened) cho 'RT-DETR (Distilled)' tại: /kaggle/working/benchmark_output/flat_distilled.yml
Đã tạo file config (flattened) cho 'RT-DETR (Baseline)' tại: /ka